In [46]:

import warnings
warnings.filterwarnings('ignore')

In [47]:
import pandas as pd

# Specify the correct encoding explicitly
urls = pd.read_excel(r"C:\Users\filip\OneDrive\Desktop\IRONHACK\Final_Project\FragranceFusion\wiki_urls.xlsx")

urls

,urls
0,A Fragrance Story
1,A Lab On Fire
2,A. N Other
3,Abercrombie & Fitch
4,Above Average Humanity
...,...
1213,3003BC
1214,4711
1215,d'Orsay
1216,pH Fragrances


In [48]:
print(urls.columns)


Index(['urls'], dtype='object')


In [49]:
urls

,urls
0,A Fragrance Story
1,A Lab On Fire
2,A. N Other
3,Abercrombie & Fitch
4,Above Average Humanity
...,...
1213,3003BC
1214,4711
1215,d'Orsay
1216,pH Fragrances


In [50]:
pip install unidecode


Note: you may need to restart the kernel to use updated packages.


In [51]:
from unidecode import unidecode

# Function to generate URL from brand name
def generate_url(brand_name):
    # Remove accents from brand name
    brand_name = unidecode(brand_name)
    # Convert brand name to lowercase
    brand_name = brand_name.lower()
    # Replace specific characters
    replacements = {
        '&': 'and',
        '(': '',
        ')': '',
        '#': 'sharp',
        '.': '',
        ' ': '-',
        "'": ''  # Remove apostrophe
    }
    for old, new in replacements.items():
        brand_name = brand_name.replace(old, new)
    # Generate URL
    url = f"https://www.wikiparfum.com/en/fragrances?filter-brand={brand_name}#catalogue"
    return url

# Convert all values in the 'urls' column to strings
urls['urls'] = urls['urls'].astype(str)

# Apply the function to create 'url' column
urls['urls'] = urls['urls'].apply(lambda x: generate_url(x) if x else '')

# Remove any duplicate URLs
urls.drop_duplicates(inplace=True)


In [52]:
brand_urls = urls

In [53]:
urls.to_csv('brand_urls.csv', index=False)

In [54]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

# Function to scrape fragrance links from a given URL
def scrape_fragrance_links(url):
    # Fetch the HTML content
    response = requests.get(url, verify=False)
    html_content = response.text
    
    # Parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find all <a> tags with href starting with "/en/fragrances/"
    fragrance_links = soup.find_all('a', href=lambda href: href and href.startswith('/en/fragrances/'))
    
    # Initialize a list to store the absolute URLs
    absolute_urls = []
    
    # Store the absolute URLs in the list
    for link in fragrance_links:
        absolute_url = urljoin(url, link.get('href'))
        absolute_urls.append(absolute_url)
    
    return absolute_urls


# Apply scraping function to the URLs
brand_urls['Absolute URLs'] = brand_urls['urls'].apply(scrape_fragrance_links)

# Explode the list of absolute URLs into separate rows
brand_urls_exploded = brand_urls.explode('Absolute URLs').reset_index(drop=True)

# Display the DataFrame
print(brand_urls_exploded)




                                                     urls  \
0       https://www.wikiparfum.com/en/fragrances?filte...   
1       https://www.wikiparfum.com/en/fragrances?filte...   
2       https://www.wikiparfum.com/en/fragrances?filte...   
3       https://www.wikiparfum.com/en/fragrances?filte...   
4       https://www.wikiparfum.com/en/fragrances?filte...   
...                                                   ...   
104485  https://www.wikiparfum.com/en/fragrances?filte...   
104486  https://www.wikiparfum.com/en/fragrances?filte...   
104487  https://www.wikiparfum.com/en/fragrances?filte...   
104488  https://www.wikiparfum.com/en/fragrances?filte...   
104489  https://www.wikiparfum.com/en/fragrances?filte...   

                                            Absolute URLs  
0          https://www.wikiparfum.com/en/fragrances/wilde  
1          https://www.wikiparfum.com/en/fragrances/wilde  
2       https://www.wikiparfum.com/en/fragrances/o-fem...  
3       https://www.wikipar

In [48]:
brand_urls_exploded.to_csv('perfume_urls_first.csv', index = False)




In [55]:
brand_urls_exploded.duplicated().sum()

74115

In [58]:
brand_urls_exploded = brand_urls_exploded.drop_duplicates()

In [59]:
brand_urls_exploded.to_csv('perfume_urls_final.csv', index = False)

In [60]:
import requests
from bs4 import BeautifulSoup
import time
time.sleep(1)  # Adding 1 second delay before making the next request

url= 'https://www.wikiparfum.com/en/fragrances/denim-original'


response = requests.get(url, verify=False)
response

<Response [200]>

In [61]:
soup = BeautifulSoup(response.content, 'html.parser')

In [62]:
# Ingredients

spans = soup.find_all(class_="leading-relaxed text-black")

# Iterate over each span and extract the text inside
ingredients = [span.get_text() for span in spans]

unique_ingredients = list(set(ingredients))

unique_ingredients

['Amber',
 'Clary Sage',
 'Petitgrain',
 'Cedarwood',
 'Vetiver',
 'Bergamot',
 'Incense / Olibanum']

In [63]:
# Name 

# Assuming you have BeautifulSoup installed and 'soup' contains the parsed HTML

# Find the span element with the specified class
span_element = soup.find(class_="relative block")

# Extract the text and remove leading/trailing whitespace
name = span_element.text.strip()

print(name)




DENIM ORIGINAL


In [64]:
# Brand

# Assuming you have BeautifulSoup installed and 'soup' contains the parsed HTML

# Find the span element with the specified class
span_element = soup.find(class_="text-base font-primaryMedium md:text-bigger3")

# Extract the text
brand = span_element.get_text(strip=True)

print(brand)


Denim


In [65]:
# Classification

# Assuming you have BeautifulSoup installed and 'soup' contains the parsed HTML

# Find the div element with the specified class
div_element = soup.find(class_="flex items-center justify-between mt-6 mb-2 md:mt-5 md:mb-2 panoramic:mt-2 panoramic:mb-0")

# Extract the text
classification = div_element.get_text(strip=True)

print(classification)



Woody Amber Classical


In [66]:
# Family and subfamily


# Assuming you have BeautifulSoup installed and 'soup' contains the parsed HTML

# Find all h6 elements with the specified class
h6_elements = soup.find_all(class_="leading-tight font-primaryMedium text-darkGray text-small2 md:text-base")

# Extract the text from each h6 element
families = [h6.get_text(strip=True) for h6 in h6_elements]

# Use set to eliminate duplicates and then convert it back to a list
unique_families = list(set(families))

# Join the unique values with a comma
family_and_subfamily = ', '.join(unique_families)

print(family_and_subfamily)



WOODY, AMBERY (ORIENTAL)


In [69]:
# Description

# Assuming you have BeautifulSoup installed and 'soup' contains the parsed HTML

# Find the div element with the specified class
div_element = soup.find(class_="hidden md:block md:mb-14")

# Extract the text
description = div_element.get_text(strip=True)

print(description)


Denim Original is a fragrance for confident men. Its spicy and unusual floral texture makes this scent enticingly rich, complex and masculine.


In [1]:
import pandas as pd


import warnings
warnings.filterwarnings('ignore')


sample = pd.read_csv(r"C:\Users\filip\OneDrive\Desktop\IRONHACK\Final_Project\FragranceFusion\perfume_urls_final_sample.csv")

sample




,urls,Absolute URLs
0,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/wilde
1,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/o-fem...
2,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/miss-...
3,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/la-vi...
4,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/karl-...
5,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/fanta...
6,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/daisy...
7,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/phant...
8,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/h24-h...
9,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/fame-...


In [2]:
# Assuming 'sample' is your DataFrame and 'Absolute URLs' is the column containing URLs
url_list = sample['Absolute URLs'].tolist()

print(url_list)


['https://www.wikiparfum.com/en/fragrances/wilde', 'https://www.wikiparfum.com/en/fragrances/o-feminin', 'https://www.wikiparfum.com/en/fragrances/miss-dior-parfum', 'https://www.wikiparfum.com/en/fragrances/la-vie-est-belle-rose-extraordinaire', 'https://www.wikiparfum.com/en/fragrances/karl-lagerfeld-pour-femme-rouge', 'https://www.wikiparfum.com/en/fragrances/fantasque', 'https://www.wikiparfum.com/en/fragrances/daisy-wild', 'https://www.wikiparfum.com/en/fragrances/phantom-intense', 'https://www.wikiparfum.com/en/fragrances/h24-herbes-vives-eau-de-parfum', 'https://www.wikiparfum.com/en/fragrances/fame-intense', 'https://www.wikiparfum.com/en/fragrances/linterdit-millesime-tubereuse-noire-2024', 'https://www.wikiparfum.com/en/fragrances/gentleman-givenchy-society-extreme', 'https://www.wikiparfum.com/en/fragrances/dance-midnight-star-2024', 'https://www.wikiparfum.com/en/fragrances/chloe-nomade-nuit-degypte', 'https://www.wikiparfum.com/en/fragrances/oud-alezan', 'https://www.wikip

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define functions for extracting information from the HTML

def extract_name(soup):
    span_element = soup.find(class_="relative block")
    return span_element.text.strip() if span_element else None

def extract_brand(soup):
    span_element = soup.find(class_="text-base font-primaryMedium md:text-bigger3")
    return span_element.get_text(strip=True) if span_element else None

def extract_ingredients(soup):
    spans = soup.find_all(class_="leading-relaxed text-black")
    ingredients = [span.get_text() for span in spans]
    unique_ingredients = list(set(ingredients))
    return ', '.join(unique_ingredients)

def extract_classification(soup):
    div_element = soup.find(class_="flex items-center justify-between mt-6 mb-2 md:mt-5 md:mb-2 panoramic:mt-2 panoramic:mb-0")
    return div_element.get_text(strip=True) if div_element else None

def extract_family_and_subfamily(soup):
    h6_elements = soup.find_all(class_="leading-tight font-primaryMedium text-darkGray text-small2 md:text-base")
    families = [h6.get_text(strip=True) for h6 in h6_elements]
    unique_families = list(set(families))
    return ', '.join(unique_families)

def extract_description(soup):
    div_element = soup.find(class_="hidden md:block md:mb-14")
    return div_element.get_text(strip=True) if div_element else None

# Function to scrape data from URLs and create DataFrame columns
def scrape_data(url_list):
    data = {'Absolute URLs': url_list}
    sample = pd.DataFrame(data)
    
    for index, row in sample.iterrows():
        url = row['Absolute URLs']
        response = requests.get(url, verify=False)  # Disable SSL verification
        soup = BeautifulSoup(response.content, 'html.parser')
        
        sample.at[index, 'Name'] = extract_name(soup)
        sample.at[index, 'Brand'] = extract_brand(soup)
        sample.at[index, 'Ingredients'] = extract_ingredients(soup)
        sample.at[index, 'Classification'] = extract_classification(soup)
        sample.at[index, 'Family_and_Subfamily'] = extract_family_and_subfamily(soup)
        sample.at[index, 'Description'] = extract_description(soup)
    
    return sample

# Assuming url_list contains the URLs

# Scrape data and create new DataFrame with extracted information
sample = scrape_data(url_list)

# Display the DataFrame with extracted information
sample






,Absolute URLs,Name,Brand,Ingredients,Classification,Family_and_Subfamily,Description
0,https://www.wikiparfum.com/en/fragrances/wilde,WILDE,Floris,"Ginger, Jasmine, Incense / Olibanum, Dianthus,...",Soft Amber Crisp Citrus,"AMBERY (ORIENTAL), SPICY","Creative expressionism of individuality, freed..."
1,https://www.wikiparfum.com/en/fragrances/o-fem...,Ô FEMININ,M. Micallef,"Jasmine, Pink Pepper, Mandarin, Rose, Patchoul...",Floral Crisp Citrus,"CITRUS, FLORAL","Sensual, carnal and modern, this chic and bohe..."
2,https://www.wikiparfum.com/en/fragrances/miss-...,MISS DIOR PARFUM,Dior,"Jasmine, Moss, Cedarwood (Alaska), Mandarin, P...",Floral Crisp Fruity,"FRUITY, FLORAL","Between strength and grace, between recklessne..."
3,https://www.wikiparfum.com/en/fragrances/la-vi...,LA VIE EST BELLE ROSE EXTRAORDINAIRE,Lancôme,"Rose, Musk, Iris / Orris, Woody Notes, Space r...",Floral Classical Rose,FLORAL,La Vie Est Belle Rose Extraordinaire is a rein...
4,https://www.wikiparfum.com/en/fragrances/karl-...,KARL LAGERFELD POUR FEMME ROUGE,Karl Lagerfeld,"Jasmine, Blackcurrant, Pear, Peach, Cashmeran ...",Floral Amber Crisp Fruity,"FLORAL, AMBERY (ORIENTAL)","Rouge is “unique, modern and captivating,” as ..."
5,https://www.wikiparfum.com/en/fragrances/fanta...,FANTASQUE,Givenchy,"Balsam (Peru), Incense (Somalia), Vanilla (Bou...",Woody Amber Classical,"AMBERY (ORIENTAL), WOODY","Eccentric and unreasonable, Fantasque twirls u..."
6,https://www.wikiparfum.com/en/fragrances/daisy...,DAISY WILD,Marc Jacobs,"Jasmine, Vetiver, Banana Flower, Sandalwood, M...",Floral Fresh Green,"CITRUS, FLORAL",Embark on an outdoors adventure with Daisy Wil...
7,https://www.wikiparfum.com/en/fragrances/phant...,PHANTOM INTENSE,Rabanne,"Moss, Rum, Cardamom, Vanilla, Lemon, Orange Bl...",Aromatic Fougère Classical,AROMATIC FOUGERE,Phantom Intense continues the evolution of the...
8,https://www.wikiparfum.com/en/fragrances/h24-h...,H24 HERBES VIVES EAU DE PARFUM,Hermès,"Earthy Notes, Aromatic Notes, Musk, Rain Notes...",Woods Crisp,"CITRUS, WOODY","Herbes Vives emerges when rain strikes, breaki..."
9,https://www.wikiparfum.com/en/fragrances/fame-...,FAME INTENSE,Rabanne,"Jasmine, Incense / Olibanum, Pink Pepper, Ylan...",Woody Amber Crisp Citrus,"FLORAL, WOODY","More irreverent, more mischievous, and more ga..."


In [4]:
perfumes = pd.read_csv(r"C:\Users\filip\OneDrive\Desktop\IRONHACK\Final_Project\FragranceFusion\perfume_urls_final.csv")
perfumes

,urls,Absolute URLs
0,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/wilde
1,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/o-fem...
2,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/miss-...
3,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/la-vi...
4,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/karl-...
...,...,...
30370,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/straw...
30371,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/merce...
30372,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/bloom...
30373,https://www.wikiparfum.com/en/fragrances?filte...,https://www.wikiparfum.com/en/fragrances/crist...


In [5]:
# Assuming 'sample' is your DataFrame and 'Absolute URLs' is the column containing URLs
url_list = perfumes['Absolute URLs'].tolist()



In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define functions for extracting information from the HTML

def extract_name(soup):
    span_element = soup.find(class_="relative block")
    return span_element.text.strip() if span_element else None

def extract_brand(soup):
    span_element = soup.find(class_="text-base font-primaryMedium md:text-bigger3")
    return span_element.get_text(strip=True) if span_element else None

def extract_ingredients(soup):
    spans = soup.find_all(class_="leading-relaxed text-black")
    ingredients = [span.get_text() for span in spans]
    unique_ingredients = list(set(ingredients))
    return ', '.join(unique_ingredients)

def extract_classification(soup):
    div_element = soup.find(class_="flex items-center justify-between mt-6 mb-2 md:mt-5 md:mb-2 panoramic:mt-2 panoramic:mb-0")
    return div_element.get_text(strip=True) if div_element else None

def extract_family_and_subfamily(soup):
    h6_elements = soup.find_all(class_="leading-tight font-primaryMedium text-darkGray text-small2 md:text-base")
    families = [h6.get_text(strip=True) for h6 in h6_elements]
    unique_families = list(set(families))
    return ', '.join(unique_families)

def extract_description(soup):
    div_element = soup.find(class_="hidden md:block md:mb-14")
    return div_element.get_text(strip=True) if div_element else None

# Function to scrape data from URLs and create DataFrame columns
def scrape_data(url_list):
    data = {'Absolute URLs': url_list}
    sample = pd.DataFrame(data)
    
    for index, row in sample.iterrows():
        url = row['Absolute URLs']
        response = requests.get(url, verify=False)  # Disable SSL verification
        soup = BeautifulSoup(response.content, 'html.parser')
        
        sample.at[index, 'Name'] = extract_name(soup)
        sample.at[index, 'Brand'] = extract_brand(soup)
        sample.at[index, 'Ingredients'] = extract_ingredients(soup)
        sample.at[index, 'Classification'] = extract_classification(soup)
        sample.at[index, 'Family_and_Subfamily'] = extract_family_and_subfamily(soup)
        sample.at[index, 'Description'] = extract_description(soup)
    
    return sample

# Assuming url_list contains the URLs

# Scrape data and create new DataFrame with extracted information
perfumes = scrape_data(url_list)

# Display the DataFrame with extracted information
perfumes(320)


KeyboardInterrupt: 

In [13]:
perfumes.to_csv('perfumes_final.csv', encoding='utf-8', index=False)


In [17]:
from unidecode import unidecode

# Assuming df is your DataFrame
from unidecode import unidecode

# Assuming df is your DataFrame
perfumes['Name'] = perfumes['Name'].apply(lambda x: unidecode(x) if x is not None else x)
perfumes['Brand'] = perfumes['Brand'].apply(lambda x: unidecode(x) if x is not None else x)
perfumes['Ingredients'] = perfumes['Ingredients'].apply(lambda x: unidecode(x) if x is not None else x)
perfumes['Classification'] = perfumes['Classification'].apply(lambda x: unidecode(x) if x is not None else x)
perfumes['Family_and_Subfamily'] = perfumes['Family_and_Subfamily'].apply(lambda x: unidecode(x) if x is not None else x)
perfumes['Description'] = perfumes['Description'].apply(lambda x: unidecode(x) if x is not None else x)





perfumes.to_csv('perfumes_final.csv', index=False)